# P4 - Architecting an LLM Integration

## Install Python Dependencies and Authenticate Azure OpenAI

In [ ]:
# Install OpenAI and datasets library
!pip install openai==0.28
%pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.3
    Uninstalling openai-1.54.3:
      Successfully uninstalled openai-1.54.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Follow the instructions provided to you in [this document](https://docs.google.com/document/d/189pgRvykpltGc3dBXVy-m34g9JEzgg4z05PYRoEIb6M/edit?tab=t.0#heading=h.oij9rqx8vz75) to get your API key and Azure endpoint.

In [ ]:
import openai

# Use your actual API Key and Endpoint
openai.api_key = "9bcZIi7ZRlZF9qvCb00ETujiDr8kpGq3fACo36yEm5xc5Envydj3JQQJ99AJACHrzpqXJ3w3AAABACOGCKto"
# openai.api_key = "1tFL5aXx54nwW76e5rgatq2UjXSEXw6VdOHhxEp5iCLpOeg7wzYwJQQJ99AJACYeBjFXJ3w3AAABACOG9mqi"  # Your API key
# Dont use the entire url!! https://[name-of-your-resource].openai.azure.com/
openai.api_base = "https://kwarraic-openai-resource.openai.azure.com/"
# openai.api_base = "https://my-openai-resource-aabdalja.openai.azure.com/"  # Your endpoint base URL
openai.api_type = "azure"
openai.api_version = "2024-08-01-preview"

# Use your deployment name
deployment_name = "khadija-gpt4-deployment"
# deployment_name = "my-gpt4-deployment"


# Call the Azure OpenAI Chat Completion API using the updated API
response = openai.ChatCompletion.create(
    engine=deployment_name,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say hello!"}
    ]
)


# Print the response
print(response.choices[0].message.content)

Hello! How can I assist you today?


### Basic Experimentation (1.5 Points)

*Before* we jump into building, your manager asks you to make sure that LLMs are suitable for the machine translation task. Implement the `get_translation` function provided below to translate non-English posts into English. Use the OpenAI API to query the `gpt-4o-mini` model. Feel free to add any helper functions and cells.

In [ ]:
# TODO: Implement Basic LLM integration
def get_translation(post: str) -> str:
    context = "Translate the following text into English." # TODO: Insert context
    # ---------------- YOUR CODE HERE ---------------- #https://oli.cmu.edu/jcourse/workbook/activity/page?context=857b38290a0001dc2ae475a176bb17ba
    response = openai.ChatCompletion.create(
        engine=deployment_name,
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": post}
        ]
    )

    translated_text = response.choices[0].message.content
    return translated_text

In [ ]:
get_translation("Hier ist dein erstes Beispiel.")

'Here is your first example.'

Similarly, you want to make sure that the LLM is capable of classifying English and non-English text. Although this is sometimes considered a solved problem within Natural Language Processing, recent research suggests that not all ML models accurately classify some English dialects and you want to avoid an embarassing screw up with NodeBB.

Optional reading [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=10313241).

In [ ]:
# TODO: Implement Basic LLM integration
def get_language(post: str) -> str:
    context = "Identify if the following text's language is in 'English','nonenglish', or 'unintelligible'" # TODO: Insert context
    # ---------------- YOUR CODE HERE ---------------- #
    response = openai.ChatCompletion.create(
        engine=deployment_name,
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": post}
        ]
    )

    classification = response.choices[0].message.content
    return classification

In [ ]:
get_language("Hier ist dein erstes Beispiel.")

'nonenglish'

Now that you have set up the basic functionality of the LLM, we would like to evaluate its performance.

### Evaluation

#### Evaluation Dataset (1.5 Points)

First, create two evaluation datasets containing example post/answer pairs, one for the translation task and one for the language classification task. Make sure each dataset has **at least 10 post-answer pairs.** Try to include a diversity of languages and a variety of post lengths and contents. You may find Google Translate, DeepL, ChatGPT, or a similar tool helpful in creating your examples.

Each dataset should be a list of dictionaries containing posts and answers. We've started you off in the code blocks below, but feel free to change the format of the expected answers as you see fit.

In [ ]:
translation_eval_set = [
    {
        "post": "Hier ist dein erstes Beispiel.",
        "expected_answer": "Here is your first example."
    },
    {
        "post": "Bonjour, comment ça va aujourd'hui?",
        "expected_answer": "Hello, how are you today?"
    },
    {
        "post": "今日は天気がいいですね。",
        "expected_answer": "The weather is nice today."
    },
    {
        "post": "¿Dónde está la biblioteca?",
        "expected_answer": "Where is the library?"
    },
    {
        "post": "Привет, как у тебя дела?",
        "expected_answer": "Hello, how are you?"
    },
    {
        "post": "Questo è un testo di esempio in italiano.",
        "expected_answer": "This is a sample text in Italian."
    },
    {
        "post": "Ich liebe es, neue Sprachen zu lernen.",
        "expected_answer": "I love learning new languages."
    },
    {
        "post": "¿Puedes ayudarme con esta tarea?",
        "expected_answer": "Can you help me with this task?"
    },
    {
        "post": "C'est une belle journée pour une promenade.",
        "expected_answer": "It's a beautiful day for a walk."
    },
    {
        "post": "Das ist eine interessante Frage.",
        "expected_answer": "That is an interesting question."
    }
]

In [ ]:
language_detection_eval_set = [
    {
        "post": "Hier ist dein erstes Beispiel.",
        "expected_answer": "nonEnglish"
    },
    {
        "post": "Bonjour, comment ça va aujourd'hui?",
        "expected_answer": "nonEnglish"
    },
    {
        "post": "The weather is really nice today.",
        "expected_answer": "English"
    },
    {
        "post": "¿Dónde está la biblioteca?",
        "expected_answer": "nonEnglish"
    },
    {
        "post": "Hello, how are you doing?",
        "expected_answer": "English"
    },
    {
        "post": "Questo è un testo di esempio in italiano.",
        "expected_answer": "nonEnglish"
    },
    {
        "post": "I love learning new languages.",
        "expected_answer": "English"
    },
    {
        "post": "C'est une belle journée pour une promenade.",
        "expected_answer": "nonEnglish"
    },
    {
        "post": "Can you help me with this task?",
        "expected_answer": "English"
    },
    {
        "post": "Привет, как у тебя дела?",
        "expected_answer": "nonEnglish"
    }
]

#### Evaluation Metrics (2 Points)
Choose a metric to measure performance of the LLM on each evaluation dataset you have created. If you would like, you can define any helper functions below in this section. You'll find the [sbert](https://www.sbert.net/docs/quickstart.html#comparing-sentence-similarities) library to be helpful.

Note that you will probably want a different evaluation metric for your language classification and translation tasks because classification has one correct answer while translation may have a spectrum of correct answers.

In [ ]:
%pip install sentence_transformers --quiet

from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def eval_single_response_translation(expected_answer: str, llm_response: str) -> float:
  '''TODO: Compares an LLM response to the expected answer from the evaluation dataset using one of the text comparison metrics.'''
  # ----------------- YOUR CODE HERE ------------------ #
  expected_embedding = model.encode(expected_answer, convert_to_tensor=True)
  response_embedding = model.encode(llm_response, convert_to_tensor=True)
  similarity_score = util.cos_sim(expected_embedding, response_embedding).item()
  return similarity_score



In [ ]:
def eval_single_response_classification(expected_answer: str, llm_response: str) -> float:
  '''TODO: Compares an LLM response to the expected answer from the evaluation dataset using one of the text comparison metrics.'''
  return 1.0 if llm_response.strip().lower() == expected_answer.strip().lower() else 0.0

In [ ]:
import time
from openai.error import OpenAIError, RateLimitError

In [ ]:
from typing import Callable

def evaluate(query_fn: Callable[[str], str], eval_fn: Callable[[str, str], float], dataset) -> float:
  '''
  TODO: Computes an aggregate score of the chosen evaluation metric across the given dataset. Calls the query_fn function to generate
  LLM outputs for each of the posts in the evaluation dataset, and calls eval_single_response to calculate the metric.
  '''
  total_score = 0.0

  for item in dataset:
      post = item['post']
      expected_answer = item['expected_answer']

      try:
        llm_response = query_fn(post)

      except RateLimitError:
        print("Rate limit exceeded, waiting for 60 seconds...")
        time.sleep(60)
        llm_response = query_fn(post)

      score = eval_fn(expected_answer, llm_response)
      total_score += score

  average_score = total_score / len(dataset)
  return average_score


Run your basic LLM implementation on the evaluation dataset and report the performance.

In [ ]:
translation_eval_score = evaluate(get_translation, eval_single_response_translation, translation_eval_set)

print(f"Translation Evaluation Score: {translation_eval_score}")

Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Translation Evaluation Score: 0.9529056787490845


In [ ]:
classification_eval_score = evaluate(get_language, eval_single_response_classification, language_detection_eval_set)

print(f"Classification Evaluation Score: {classification_eval_score}")

Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Classification Evaluation Score: 1.0


### Querying for Success

#### Evaluation Set (1.5 Points)
Now that you're confident in the LLM's translation and language detection abilities, you want to begin work on your actual queries. But first, create an evaluation set called `translation_eval_set` that includes at least 15 English posts, at least 15 non-English posts, and at least 5 unintelligible or malformed posts. You may reuse your posts from earlier tasks. Your output format should be a tuple with a boolean indicating whether the post is in English and a string with the translation of a non-English post into English. Use your best judgement about what string to expect when the post is already in English.

**It is very important that you do not make ANY assumptions about the contents of the submitted post.**

In [ ]:
complete_eval_set = [

    {"post": "Hier ist dein erstes Beispiel.", "expected_answer": (False, "Here is your first example.")},
    {"post": "¿Cómo estás hoy?", "expected_answer": (False, "How are you today?")},
    {"post": "C'est la vie.", "expected_answer": (False, "That's life.")},
    {"post": "Quanto costa?", "expected_answer": (False, "How much does it cost?")},
    {"post": "今日はどうですか。", "expected_answer": (False, "How are you today?")},
    {"post": "Это было великолепно!", "expected_answer": (False, "That was magnificent!")},
    {"post": "Où est la bibliothèque?", "expected_answer": (False, "Where is the library?")},
    {"post": "Todo lo que necesitas es amor.", "expected_answer": (False, "All you need is love.")},
    {"post": "La esperanza es lo último que se pierde.", "expected_answer": (False, "Hope is the last thing ever lost.")},
    {"post": "私の猫が好きです。", "expected_answer": (False, "I like my cat.")},
    {"post": "Il fait très froid aujourd'hui.", "expected_answer": (False, "It is very cold today.")},
    {"post": "何時ですか?", "expected_answer": (False, "What time is it?")},
    {"post": "Le prix de la liberté est la vigilance éternelle.", "expected_answer": (False, "The price of freedom is eternal vigilance.")},
    {"post": "¿Dónde está el baño?", "expected_answer": (False, "Where is the bathroom?")},
    {"post": "你今天好吗？", "expected_answer": (False, "How are you today?")},


    {"post": "This is a simple test.", "expected_answer": (True, "Already English")},
    {"post": "What time does the store close?", "expected_answer": (True, "Already English")},
    {"post": "I need some help with my project.", "expected_answer": (True, "Already English")},
    {"post": "How far is it from here?", "expected_answer": (True, "Already English")},
    {"post": "Can you recommend a good book?", "expected_answer": (True, "Already English")},
    {"post": "It's raining cats and dogs today!", "expected_answer": (True, "Already English")},
    {"post": "I'm learning to play the guitar.", "expected_answer": (True, "Already English")},
    {"post": "The quick brown fox jumps over the lazy dog.", "expected_answer": (True, "Already English")},
    {"post": "All is fair in love and war.", "expected_answer": (True, "Already English")},
    {"post": "He is an old friend of mine.", "expected_answer": (True, "Already English")},
    {"post": "Where can I find the nearest gas station?", "expected_answer": (True, "Already English")},
    {"post": "Who won the game last night?", "expected_answer": (True, "Already English")},
    {"post": "Is there a vegetarian option on the menu?", "expected_answer": (True, "Already English")},
    {"post": "What's the weather like tomorrow?", "expected_answer": (True, "Already English")},
    {"post": "Can you help me with my homework?", "expected_answer": (True, "Already English")},



    {"post": "sdgkl fdkg ldfkgj dflkgj", "expected_answer": (False, "Unintelligible")},
    {"post": "12345 67890", "expected_answer": (False, "Unintelligible")},
    {"post": "!@#$%^&*()_+", "expected_answer": (False, "Unintelligible")},
    {"post": "??? ??? ???", "expected_answer": (False, "Unintelligible")},
    {"post": "无无无 何何何", "expected_answer": (False, "Unintelligible")}
]


#### Implementation (3.5 Points)

Implement the `query_llm` function that takes in a NodeBB post and outputs a response in your expected format. You may wish to experiment with a variety of querying strategies, including those discussed in lecture.

In [ ]:
def query_llm(post: str) -> tuple[bool, str]:

  # ----------------- YOUR CODE HERE ------------------ #
  language = get_language(post)
  if language.lower().strip() == "english":
    return (True, "Already English")
  elif language.lower().strip() == "nonenglish":
    time.sleep(60)
    translation = get_translation(post)
    return (False, translation)
  else:
    return (False, "Unintelligible")

In [ ]:
query_llm("Hier ist dein erstes Beispiel.")

(False, 'Here is your first example.')

You may also need a new evaluation function for this format. Feel free to call your existing evaluation functions or create a completely new one. This will be scored with the other evaluation functions above.

In [ ]:
def eval_single_response_complete(expected_answer: tuple[bool, str], llm_response: tuple[bool, str]) -> float:
  '''TODO: Compares an LLM response to the expected answer from the evaluation dataset using one of the text comparison metrics.'''
  # ----------------- YOUR CODE HERE ------------------ #
  expected_lang, expected_text = expected_answer
  response_lang, response_text = llm_response
  lang_score = 1.0 if expected_lang == response_lang else 0.0
  expected_embedding = model.encode(expected_text, convert_to_tensor=True)
  response_embedding = model.encode(response_text, convert_to_tensor=True)
  similarity_score = util.cos_sim(expected_embedding, response_embedding).item()
  combined_score = 0.5 * lang_score + 0.5 * similarity_score
  return combined_score


In [ ]:
eval_score = evaluate(query_llm, eval_single_response_complete, complete_eval_set)

print(f"Evaluation Score: {eval_score}")

Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit exceeded, waiting for 60 seconds...
Rate limit ex

##Mocking up a Storm
Because NodeBB now relies on an external LLM, we have introduced new risks into our platform. Even if our integration works perfectly today, Microsoft could decide to change their model tomorrow. It is our responsibility to make sure that changes to the model do not jeopardize our platform. For this task, you will modify your `query_llm` function from above to fail gracefully in the event of unexpected model responses or states and test your new function with both normal and unexpected model responses.

### Handle Errors (3 points)

Copy your `query_llm` function from above. Modify it so that you check whether your model's output is in your expected format. If it is not in the correct format, your new function should allow NodeBB to handle the error gracefully (that is, NodeBB should not break and should continue to provide an okay user experience). In the provided space below, please briefly describe how your function responds to poorly formatted model outputs and how you plan to use these responses in your NodeBB integration.

Note that we do not expect you to verify that the language classification or translation is correct, only that it could be correct for some input. As with before, we expect your querying function to be robust in the face of any textual input.

In [ ]:
def query_llm_robust(post: str) -> tuple[bool, str]:
  '''
  TODO: Implement this
  '''
  try:
        time.sleep(60)
        language = get_language(post)

        if isinstance(language, str) and language.isalpha():
            if language.lower().strip() == "english":
              return (True, "Already English")

            if language.lower().strip() == "nonenglish":
              time.sleep(60)
              translation = get_translation(post)
              time.sleep(60)
              return (False, translation)

            if language.lower().strip() == "unintelligible":
              return (False, "Unintelligible")

        return (False, "Unavailable")

  except Exception as e:
      return (False, "Error")


BRIEFLY EXPLAIN YOUR TEAM'S STRATEGY FOR HANDLING ERRORS HERE

The strategy for handling errors in query_llm_robust is to check if the model’s output matches the expected format. If the response is not in this format, or if an error occurs during the API call, the function provides a fallback response with a default message. This ensures that NodeBB remains stable and users receive a consistent experience, even when the model output is unexpected or an error occurs. This approach isolates NodeBB from external model changes and allows for easy troubleshooting if issues arise.

### Test Robustness (2 points)

Mocking is a common strategy in software engineering for unit testing. It allows us to simulate the behavior of an object, such as a microservice or API call, which can be helpful when the object is not yet complete, slow, nondeterministic or has rare states that we want to test (such as a service being down). In this case, we will mock the behavior of the LLM so that it will simulate the LLM returning unexpected or malformed results.

You should refer to [this](https://docs.python.org/3/library/unittest.mock.html) Python documentation as you go through this task.

You may (optionally) read more about mocking in Python [here](https://www.fugue.co/blog/2016-02-11-python-mocking-101).

For this task, you should write at least four tests that deal with unexpected model behavior.

In [ ]:
%pip install --quiet pytest ipytest pytest-mock mock

In [ ]:
from mock import patch

@patch.object(openai.ChatCompletion, 'create')
def test_unexpected_language(mocker):
  # we mock the model's response to return a random message
  mocker.return_value.choices[0].message.content = "I don't understand your request"

  # TODO assert the expected behavior
  assert query_llm_robust("Hier ist dein erstes Beispiel.") == (False, "Unavailable")

@patch.object(openai.ChatCompletion, 'create')
def test_unexpected_return(mocker):
  # we mock the model's response to return a random message
  mocker.return_value.choices[0].message.content = "dhkajshfljaelh"

  # TODO assert the expected behavior
  assert query_llm_robust("Hier ist dein erstes Beispiel.") == (False, "Unavailable")

@patch.object(openai.ChatCompletion, 'create')
def test_unexpected_char(mocker):
  # we mock the model's response to return a random message
  mocker.return_value.choices[0].message.content = "#@&^$#&^@%$&"

  # TODO assert the expected behavior
  assert query_llm_robust("Hier ist dein erstes Beispiel.") == (False, "Unavailable")

@patch.object(openai.ChatCompletion, 'create')
def test_unexpected_random(mocker):
  # we mock the model's response to return a random message
  mocker.return_value.choices[0].message.content = "ءايخاخثبخثم"

  # TODO assert the expected behavior
  assert query_llm_robust("Hier ist dein erstes Beispiel.") == (False, "Unavailable")

@patch.object(openai.ChatCompletion, 'create')
def test_unexpected_empty(mocker):
  # we mock the model's response to return a random message
  mocker.return_value.choices[0].message.content = "."
  # TODO assert the expected behavior
  assert query_llm_robust("Hier ist dein erstes Beispiel.") == (False, "Unavailable")


Run the tests.

In [ ]:
import ipytest
ipytest.run('-vv')

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-8.3.3, pluggy-1.5.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: mock-3.14.0, typeguard-4.4.1, anyio-3.7.1
collecting ... collected 5 items

t_b484825051a14555a5cf3f07a687745a.py::test_unexpected_language <- <ipython-input-29-04c41653cf41> PASSED [ 20%]
t_b484825051a14555a5cf3f07a687745a.py::test_unexpected_return <- <ipython-input-29-04c41653cf41> PASSED [ 40%]
t_b484825051a14555a5cf3f07a687745a.py::test_unexpected_char <- <ipython-input-29-04c41653cf41> PASSED [ 60%]
t_b484825051a14555a5cf3f07a687745a.py::test_unexpected_random <- <ipython-input-29-04c41653cf41> PASSED [ 80%]
t_b484825051a14555a5cf3f07a687745a.py::test_unexpected_empty <- <ipython-input-29-04c41653cf41> PASSED [100%]

======================================== 5 passed in 0.07s =========================================


<ExitCode.OK: 0>

Finally, we want to make sure that our modified function still works for normal inputs, so let's test it to find out!

In [ ]:
query_llm_robust("Hier ist dein erstes Beispiel.")

(False, 'Here is your first example.')

In [ ]:
eval_score = evaluate(query_llm_robust, eval_single_response_complete, complete_eval_set)

print(f"Evaluation Score: {eval_score}")

Evaluation Score: 0.9845897785254887
